In [1]:
!pip install pymorphy2

In [2]:
!pip install —upgrade pandas==1.1.5

ERROR: Invalid requirement: '—upgrade'


In [3]:
!pip install —upgrade numpy==1.19.5

ERROR: Invalid requirement: '—upgrade'


In [4]:
! pip uninstall pyLDAvis

Uninstalling pyLDAvis-3.3.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pyLDAvis-3.3.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pyLDAvis/*
Proceed (y/n)? y
  Successfully uninstalled pyLDAvis-3.3.1


In [5]:
! pip install pyLDAvis

Processing /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2/pyLDAvis-3.3.1-cp37-none-any.whl


In [6]:
import json
import nltk
import string
from nltk.corpus import stopwords
import re
import pymorphy2
import gensim.corpora as corpora
import pyLDAvis.gensim_models
import pyLDAvis
import pickle
import os

nltk.download('stopwords')

analyzer = pymorphy2.MorphAnalyzer()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

### Предобработка

Подгружаем словарь, где ключи - заголовки документов, ключи - токены этого документа в начальной форме

In [7]:
with open("new_corpus.json") as f:
  corpus = json.load(f)

In [8]:
corpus_arr = [] # здесь мы будем хранить нормализованные документы, состоит из corpus_arr_little
for doc in corpus.values():
  corpus_arr_little = [] # массив для каждого нормализованного документа
  array = doc.split()
  for word in array:
    normal_word = analyzer.parse(word)[0]
    corpus_arr_little.append(normal_word.normal_form)
  corpus_arr.append(corpus_arr_little)

In [9]:
def del_stops(word):
    """
    Функция удаления стоп-слов
    """
    punct = string.punctuation + "—" + "«" + "»" + "[" + "]" + "{" + "}" 
    if (word in punct) or (word == '{\\displaystyle') or (analyzer.parse(word)[0].tag.POS in ['CONJ', 'PRCL', 'INTJ', 'PREP', 'NPRO', 'NUMR']) or (len(word) <= 2) or ('x' in word):
      return False

    else:
    # Удаление служебных частей речи и личных местоимений
      word = word.replace(',','')
      stops = stopwords.words("english") + stopwords.words("russian") + ["это", "который", "наш", "мочь", "год", 
                                            "такой", "знать", "мы", "свой", "один", "другой", "хотеть",
                                            "человек", "всё", "все", "весь", "очень", "думать", "нужно",
                                            "большой", "время", "использовать", "говорить", "сказать",
                                            "иметь", "сделать", "первый", "каждый", "день", "её", "ваш",
                                            "стать", "больший", "ваше", "день", "самый", "понять",
                                            "просто", "ещё", "проблема", "также", "например", "м", "c", "cos", "sin", "x"]
      if word in stops:
        return False
      else:
        return word

In [10]:
corpus = []
for i in corpus_arr:
  clean_arr_words = []
  for j in i:
    new_form = del_stops(j)
    if not new_form == False:
      clean_arr_words.append(new_form)
  corpus.append(clean_arr_words)

### Работа с LDA

In [11]:
id2word = corpora.Dictionary(corpus)

In [12]:
docs = corpus
corpus = [id2word.doc2bow(doc) for doc in docs]

In [13]:
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 4), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)]


In [30]:
from gensim.models import LdaMulticore
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=4)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [31]:
lda_model.print_topics()

[(0,
  '0.009*"город" + 0.004*"россия" + 0.004*"антициклон" + 0.003*"российский" + 0.003*"городской" + 0.003*"область" + 0.003*"война" + 0.003*"часть" + 0.003*"центр" + 0.003*"архангельский"'),
 (1,
  '0.009*"город" + 0.007*"район" + 0.007*"архангельский" + 0.007*"российский" + 0.005*"область" + 0.005*"центр" + 0.005*"состав" + 0.005*"часть" + 0.004*"административный" + 0.004*"образование"'),
 (2,
  '0.006*"архангельский" + 0.006*"город" + 0.005*"являться" + 0.005*"область" + 0.004*"российский" + 0.004*"крупный" + 0.004*"территория" + 0.004*"совет" + 0.003*"центр" + 0.003*"северный"'),
 (3,
  '0.006*"город" + 0.006*"северный" + 0.005*"центр" + 0.005*"являться" + 0.004*"крупный" + 0.004*"область" + 0.003*"государство" + 0.003*"россия" + 0.003*"река" + 0.003*"население"')]

In [32]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.006918 -0.028354       1        1  34.091395
3      0.015848 -0.009521       2        1  23.090373
1     -0.041751  0.006124       3        1  21.736367
0      0.018984  0.031750       4        1  21.081865, topic_info=            Term       Freq      Total Category  logprob  loglift
29         район  42.000000  42.000000  Default  30.0000  30.0000
1211  балтийский  13.000000  13.000000  Default  29.0000  29.0000
589   антициклон  17.000000  17.000000  Default  28.0000  28.0000
13         город  93.000000  93.000000  Default  27.0000  27.0000
399        совет  20.000000  20.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
79    территория   5.260143  36.719336   Topic4  -6.2745  -0.3864
29         район   5.046023  42.335311   Topic4  -6.3161  -0.5703
817        север   4.649232  21.128136   Topic4  -6.3980   0.0429
404         ссср   4.536474  21.187329   Topic4  -6.4225   0.0155
349       остров   4.529301  20.318776   Topic4  -6.4241   0.0558

[336 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3309      4  0.733337      1972
2000      1  0.744713      1993
2000      4  0.248238      1993
3         3  0.524915      2016
4         3  0.525073     668-р
...     ...       ...       ...
3011      2  0.486086  эстуарий
291       1  0.435848  являться
291       2  0.290565  являться
291       3  0.163443  являться
291       4  0.127122  являться

[596 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

In [29]:
LDAvis_prepared.topic_order

[3, 5, 1, 2, 4]

### Вывод
Только при маленьком количестве тем (4 и меньше) круги не пересекаются.
Во всех темах преобладают слова темы города, севера, государства и административных единиц, что очень хорошо отображает содержание корпуса